# MVP – Camada Silver (Padronização e estruturação dos dados)

Esta etapa implementa a camada Silver do pipeline do MVP, responsável por:

- consumir as tabelas estruturadas da camada Bronze,
- padronizar nomes, tipos e estruturas das entidades clínicas,
- filtrar e organizar os registros relevantes ao escopo do MVP (ex.: internações),
- garantir integridade referencial mínima entre pacientes, internações e diagnósticos,
- documentar e inspecionar inconsistências sem aplicar correções automáticas,
- persistir as tabelas consolidadas como Delta tables no schema silver,
- preparar os dados para a modelagem dimensional e o cálculo de indicadores na camada Gold.


In [0]:
# Imports globais utilizados em todo o notebook

from pyspark.sql.functions import (
    col, current_timestamp, datediff,
    min as spark_min, max as spark_max
)


In [0]:
%sql
USE CATALOG mvp_engenharia_de_dados;
USE SCHEMA bronze;


In [0]:
# Ler a tabela patients_bronze
df_pat_bronze = spark.table("patients_bronze")

df_pat_bronze.printSchema()
display(df_pat_bronze.limit(10))



root
 |-- Id: string (nullable = true)
 |-- BIRTHDATE: date (nullable = true)
 |-- DEATHDATE: date (nullable = true)
 |-- SSN: string (nullable = true)
 |-- DRIVERS: string (nullable = true)
 |-- PASSPORT: string (nullable = true)
 |-- PREFIX: string (nullable = true)
 |-- FIRST: string (nullable = true)
 |-- MIDDLE: string (nullable = true)
 |-- LAST: string (nullable = true)
 |-- SUFFIX: string (nullable = true)
 |-- MAIDEN: string (nullable = true)
 |-- MARITAL: string (nullable = true)
 |-- RACE: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- BIRTHPLACE: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTY: string (nullable = true)
 |-- FIPS: integer (nullable = true)
 |-- ZIP: integer (nullable = true)
 |-- LAT: double (nullable = true)
 |-- LON: double (nullable = true)
 |-- HEALTHCARE_EXPENSES: double (nullable = true)
 |--

Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,MIDDLE,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
cfb6239f-9bc0-8b4f-58cb-0789a9da2f7f,2016-10-24,null,999-90-8784,null,null,null,Margareta320,Aracelis412,Yundt842,null,null,null,white,nonhispanic,F,Amherst Massachusetts US,694 White Wynd,Charlton,Massachusetts,Worcester County,null,0,42.152916728037894,-71.93639278709503,11793.44,9108.57,141848
9c43b243-6ce0-bb9a-52f1-ae426dc840a1,2002-12-15,null,999-65-8958,S99992259,X88257069X,Ms.,Marlyn309,Trudi580,Reichel38,null,null,null,white,nonhispanic,F,Lawrence Massachusetts US,997 Mante Wall Apt 36,Plainville,Massachusetts,Norfolk County,null,0,41.99993136128743,-71.34072771832133,113194.6,95703.64,123743
f044c345-b58f-4304-8120-f16ee25c3552,2004-06-15,null,999-25-4282,S99952238,X20988121X,Ms.,Keith571,null,Stokes453,null,null,null,white,nonhispanic,F,Chelsea Massachusetts US,536 Corkery Wynd,Chicopee,Massachusetts,Hampden County,25013,1013,42.17341424796212,-72.58680299375206,125715.35,174703.21,45223
04a6d0fd-dd6d-94a9-984a-36907ffd5dc9,1973-10-08,null,999-93-2159,S99973296,X41513019X,Mrs.,Ronda430,Ka422,Boyer713,null,Bosco882,M,black,nonhispanic,F,Monson Center Massachusetts US,152 Cremin Well Apt 5,Woburn,Massachusetts,Middlesex County,25017,1890,42.52811910518509,-71.11147783658413,609653.19,131054.58,102242
3e9dd1f5-c9fc-4015-648e-0c79efb02594,1972-02-26,null,999-39-6039,S99945641,X43098897X,Mrs.,Angelic427,Renata373,Vandervort697,null,Hickle134,M,white,nonhispanic,F,Russell Massachusetts US,128 Kuvalis Terrace Suite 52,Oak Bluffs,Massachusetts,Dukes County,null,0,41.42922569053695,-70.5329668542055,932534.22,299132.62,103003
6276e83f-636c-70e8-aaae-27a74d204ee3,1988-09-17,null,999-36-4670,S99991265,X50821604X,Mr.,Derrick232,Terrance440,Witting912,null,null,S,white,nonhispanic,M,Pinehurst Massachusetts US,592 Ortiz Route Apt 17,New Bedford,Massachusetts,Bristol County,25005,2745,41.71755587259173,-70.85194405294612,145387.33,485.96,81651
d1a76952-bd8a-1304-6a25-5a68107ddef5,2015-12-08,null,999-80-9745,null,null,null,Dennise990,Veronika907,Crist667,null,null,null,hawaiian,hispanic,F,Springfield Massachusetts US,799 Funk Well,Groveland,Massachusetts,Essex County,null,0,42.78087990609578,-71.0017664845296,28457.77,0.0,114520
9f139e0d-3ee7-dcc8-4c15-b19f9b6077c7,2020-03-13,null,999-17-3983,null,null,null,Harrison106,null,Goodwin327,null,null,null,white,nonhispanic,M,Worcester Massachusetts US,1061 Cartwright Row Apt 94,Brookline,Massachusetts,Norfolk County,25021,2446,42.334815815370845,-71.14667329222318,4761.9,13002.5,189808
fcc5cb15-c638-1ed3-664c-c69aedd94e50,2004-12-19,null,999-35-4664,S99997944,X40905897X,Mr.,Emory494,Angel97,Bogisich202,null,null,null,white,nonhispanic,M,Fall River Massachusetts US,725 Gleichner Parade Suite 95,Mansfield,Massachusetts,Bristol County,null,0,42.01503024218797,-71.18379267568328,56297.25,10387.34,51036
89442147-a7d9-9a15-6264-f038fd11ac81,2014-06-16,null,999-32-3369,null,null,null,Danyelle408,Joanne343,Durgan499,null,null,null,asian,nonhispanic,F,Shanghai Shanghai Municipality CN,706 Koepp Corner,Wellesley,Massachusetts,Norfolk County,25021,2457,42.303261336134206,-71.27559312143254,11754.95,7788.86,531025


## Criaçao da tabela `patients_silver`

A tabela `patients_silver` consolida os principais atributos demográficos do paciente.

Critérios adotados:
- Seleção explícita de colunas relevantes para o MVP
- Padronização de nomes
- Um registro por paciente


In [0]:

# Transformaçoes da tabela patients_bronze para criar a tabela patients_silver
df_patients_silver = (
    df_pat_bronze
    .dropDuplicates(["Id"])                              # remove duplicados
    .withColumnRenamed("Id", "patient_id")              # padroniza nome
    .withColumn("BIRTHDATE", col("BIRTHDATE").cast("date"))
    .withColumn("DEATHDATE", col("DEATHDATE").cast("date"))
    .select(                                             # selecao de colunas
        "patient_id",
        "BIRTHDATE",
        "DEATHDATE",
        "GENDER",
        "RACE",
        "ETHNICITY"
    )
)

df_patients_silver.printSchema()
display(df_patients_silver.limit(10))


root
 |-- patient_id: string (nullable = true)
 |-- BIRTHDATE: date (nullable = true)
 |-- DEATHDATE: date (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- RACE: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)



patient_id,BIRTHDATE,DEATHDATE,GENDER,RACE,ETHNICITY
cfb6239f-9bc0-8b4f-58cb-0789a9da2f7f,2016-10-24,null,F,white,nonhispanic
9c43b243-6ce0-bb9a-52f1-ae426dc840a1,2002-12-15,null,F,white,nonhispanic
f044c345-b58f-4304-8120-f16ee25c3552,2004-06-15,null,F,white,nonhispanic
04a6d0fd-dd6d-94a9-984a-36907ffd5dc9,1973-10-08,null,F,black,nonhispanic
3e9dd1f5-c9fc-4015-648e-0c79efb02594,1972-02-26,null,F,white,nonhispanic
6276e83f-636c-70e8-aaae-27a74d204ee3,1988-09-17,null,M,white,nonhispanic
d1a76952-bd8a-1304-6a25-5a68107ddef5,2015-12-08,null,F,hawaiian,hispanic
9f139e0d-3ee7-dcc8-4c15-b19f9b6077c7,2020-03-13,null,M,white,nonhispanic
fcc5cb15-c638-1ed3-664c-c69aedd94e50,2004-12-19,null,M,white,nonhispanic
89442147-a7d9-9a15-6264-f038fd11ac81,2014-06-16,null,F,asian,nonhispanic


**Chacagem da qualidade dos dados da tabela patients_silver**

Para assegurar que as informações demográficas essenciais estão completas, coerentes e prontas para uso na modelagem analítica da camada Gold. As validações incluem: esquema e tipos de dados, duplicatas, presença de valores nulos, consistência das datas de nascimento e óbito, e plausibilidade da idade.

In [0]:
print("====================================================")
print("      QUALITY CHECK — patients_silver")
print("====================================================")

# -----------------------------------------------------
# 1) SCHEMA
# -----------------------------------------------------
print("\n>> 1. Schema do patients_silver:")
df_patients_silver.printSchema()


# -----------------------------------------------------
# 2) DUPLICATAS POR patient_id
# -----------------------------------------------------
print("\n>> 2. Checando duplicatas por patient_id:")

total = df_patients_silver.count()
unique = df_patients_silver.dropDuplicates(["patient_id"]).count()

print(f"Total de registros:      {total}")
print(f"Registros únicos:        {unique}")
print(f"Duplicatas encontradas:  {total - unique}")


# -----------------------------------------------------
# 3) NULOS NAS PRINCIPAIS COLUNAS
# -----------------------------------------------------
print("\n>> 3. Nulos em BIRTHDATE e DEATHDATE:")

birth_nulls = df_patients_silver.filter(col("BIRTHDATE").isNull()).count()
death_nulls = df_patients_silver.filter(col("DEATHDATE").isNull()).count()

print(f"BIRTHDATE nulos: {birth_nulls}")
print(f"DEATHDATE nulos: {death_nulls} (normal)")


# -----------------------------------------------------
# 4) DATAS NO FUTURO
# -----------------------------------------------------
print("\n>> 4. Datas no futuro:")

birth_in_future = df_patients_silver.filter(col("BIRTHDATE") > current_date()).count()
death_in_future = df_patients_silver.filter(col("DEATHDATE") > current_date()).count()

print(f"BIRTHDATE no futuro: {birth_in_future}")
print(f"DEATHDATE no futuro: {death_in_future}")


# -----------------------------------------------------
# 5) INCONSISTÊNCIA: DEATHDATE < BIRTHDATE
# -----------------------------------------------------
print("\n>> 5. Inconsistência: DEATHDATE anterior a BIRTHDATE:")

death_before_birth = df_patients_silver.filter(
    col("DEATHDATE").isNotNull() & (col("DEATHDATE") < col("BIRTHDATE"))
).count()

print(f"Registros inconsistentes: {death_before_birth}")


# -----------------------------------------------------
# 6) IDADE NEGATIVA E IDADE IMPROVÁVEL ALTA
# -----------------------------------------------------
print("\n>> 6. Validação de idade (negativa ou > 120 anos):")

# idade hoje
age_years = datediff(current_date(), col("BIRTHDATE")) / 365.25

age_negative = df_patients_silver.filter(age_years < 0).count()
age_too_high = df_patients_silver.filter(age_years > 120).count()

print(f"Pacientes com idade negativa:  {age_negative}")
print(f"Pacientes com idade > 120 anos: {age_too_high}")


# -----------------------------------------------------
# 7) INTERVALOS DE DATAS (mínimo e máximo)
# -----------------------------------------------------
print("\n>> 7. Intervalo de datas no dataset:")

df_patients_silver.agg(
    spark_min("BIRTHDATE").alias("min_birthdate"),
    spark_max("BIRTHDATE").alias("max_birthdate"),
    spark_min("DEATHDATE").alias("min_deathdate"),
    spark_max("DEATHDATE").alias("max_deathdate")
).show()

print("\n### QUALITY CHECK — patients_silver COMPLETO ###")


      QUALITY CHECK — patients_silver

>> 1. Schema do patients_silver:
root
 |-- patient_id: string (nullable = true)
 |-- BIRTHDATE: date (nullable = true)
 |-- DEATHDATE: date (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- RACE: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)


>> 2. Checando duplicatas por patient_id:
Total de registros:      9037
Registros únicos:        9037
Duplicatas encontradas:  0

>> 3. Nulos em BIRTHDATE e DEATHDATE:
BIRTHDATE nulos: 0
DEATHDATE nulos: 7918 (normal)

>> 4. Datas no futuro:
BIRTHDATE no futuro: 0
DEATHDATE no futuro: 0

>> 5. Inconsistência: DEATHDATE anterior a BIRTHDATE:
Registros inconsistentes: 0

>> 6. Validação de idade (negativa ou > 120 anos):
Pacientes com idade negativa:  0
Pacientes com idade > 120 anos: 0

>> 7. Intervalo de datas no dataset:
+-------------+-------------+-------------+-------------+
|min_birthdate|max_birthdate|min_deathdate|max_deathdate|
+-------------+-------------+-------------

**Observações da checagem de dados da tabela `patients_silver`**

Os resultados mostraram que a tabela patients_silver está íntegra: não foram encontradas duplicatas ou datas inconsistentes, não há datas no futuro, e todas as idades são plausíveis. A ausência de nulos em BIRTHDATE garante confiabilidade para cálculos futuros de idade e estratificação demográfica. O alto número de nulos em DEATHDATE é esperado, pois a maioria dos pacientes está viva. Assim, os dados estão adequados para alimentar a dimensão de pacientes na camada Gold.

## Criação da tabela `encounters_silver`

A tabela `encounters_silver` representa as internações hospitalares utilizadas no escopo do MVP.

Critérios adotados:
- Seleção apenas de encontros do tipo inpatient - referentes às internaçoes hospitalares
- Padronização dos nomes das colunas
- Manutenção das datas e horas completas de admissão e alta
- Utilização de `reason_description` como motivo principal da internação
- Um registro por internação


In [0]:
# Leitura da tabela encouters_bronze
df_enc_bronze = spark.table("encounters_bronze")

df_enc_bronze.printSchema()
display(df_enc_bronze.limit(5))


root
 |-- Id: string (nullable = true)
 |-- START: timestamp (nullable = true)
 |-- STOP: timestamp (nullable = true)
 |-- PATIENT: string (nullable = true)
 |-- ORGANIZATION: string (nullable = true)
 |-- PROVIDER: string (nullable = true)
 |-- PAYER: string (nullable = true)
 |-- ENCOUNTERCLASS: string (nullable = true)
 |-- CODE: string (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- BASE_ENCOUNTER_COST: string (nullable = true)
 |-- TOTAL_CLAIM_COST: string (nullable = true)
 |-- PAYER_COVERAGE: string (nullable = true)
 |-- REASONCODE: string (nullable = true)
 |-- REASONDESCRIPTION: string (nullable = true)



Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
cfb6239f-9bc0-8b4f-610f-e84dd785f7d8,2016-10-24T06:53:58.000Z,2016-10-24T07:08:58.000Z,cfb6239f-9bc0-8b4f-58cb-0789a9da2f7f,4ff8b164-4cf5-3ab4-b0f2-7ce8fda920e5,75742a69-e63f-39e0-bbe8-3634bd82b239,d31fccc3-1767-390d-966a-22a5156f4219,wellness,410620009,Well child visit (procedure),136.80,1034.05,0.00,null,null
cfb6239f-9bc0-8b4f-beed-fc61be2d7916,2016-11-28T06:53:58.000Z,2016-11-28T07:08:58.000Z,cfb6239f-9bc0-8b4f-58cb-0789a9da2f7f,4ff8b164-4cf5-3ab4-b0f2-7ce8fda920e5,75742a69-e63f-39e0-bbe8-3634bd82b239,d31fccc3-1767-390d-966a-22a5156f4219,wellness,410620009,Well child visit (procedure),136.80,272.80,0.00,null,null
cfb6239f-9bc0-8b4f-297a-c280fd930088,2017-01-30T06:53:58.000Z,2017-01-30T07:08:58.000Z,cfb6239f-9bc0-8b4f-58cb-0789a9da2f7f,4ff8b164-4cf5-3ab4-b0f2-7ce8fda920e5,75742a69-e63f-39e0-bbe8-3634bd82b239,d31fccc3-1767-390d-966a-22a5156f4219,wellness,410620009,Well child visit (procedure),136.80,1096.58,756.34,null,null
cfb6239f-9bc0-8b4f-9220-8a9883c75e99,2017-04-03T06:53:58.000Z,2017-04-03T07:08:58.000Z,cfb6239f-9bc0-8b4f-58cb-0789a9da2f7f,4ff8b164-4cf5-3ab4-b0f2-7ce8fda920e5,75742a69-e63f-39e0-bbe8-3634bd82b239,d31fccc3-1767-390d-966a-22a5156f4219,wellness,410620009,Well child visit (procedure),136.80,816.80,653.44,null,null
cfb6239f-9bc0-8b4f-8be5-74d7ac2182fa,2017-07-03T06:53:58.000Z,2017-07-03T07:08:58.000Z,cfb6239f-9bc0-8b4f-58cb-0789a9da2f7f,4ff8b164-4cf5-3ab4-b0f2-7ce8fda920e5,75742a69-e63f-39e0-bbe8-3634bd82b239,d31fccc3-1767-390d-966a-22a5156f4219,wellness,410620009,Well child visit (procedure),136.80,1150.65,920.52,null,null


In [0]:
df_enc_bronze.select("ENCOUNTERCLASS").distinct().show()


+--------------------+
|      ENCOUNTERCLASS|
+--------------------+
|             virtual|
|             hospice|
|           emergency|
|            wellness|
|                home|
|          outpatient|
|          ambulatory|
|           inpatient|
|          urgentcare|
|                 snf|
|                NULL|
|e674f90b-62b3-36d...|
+--------------------+



Transformacoes da tabela encounters_bronze para criar a tabela encounters_silver

In [0]:
# Transfomações da tabela encouters_bronze para criar a tabela encounters_silver

df_enc_silver = (
    df_enc_bronze
      # 1) só internações
      .filter(col("ENCOUNTERCLASS") == "inpatient")
      
      # 2) remove duplicados
      .dropDuplicates(["Id"])
      
      # 3) renomeia colunas-chave
      .withColumnRenamed("Id", "encounter_id")
      .withColumnRenamed("PATIENT", "patient_id")
      
      # 4) garante tipos de data/hora
      .withColumn("START", col("START").cast("timestamp"))
      .withColumn("STOP",  col("STOP").cast("timestamp"))
      
      # 5) Seleçao de colunas
      .select(
          "encounter_id",
          "patient_id",
          "START",
          "STOP",
          "REASONCODE",
          "REASONDESCRIPTION"
      )
)

df_enc_silver.printSchema()
display(df_enc_silver.limit(10))


root
 |-- encounter_id: string (nullable = true)
 |-- patient_id: string (nullable = true)
 |-- START: timestamp (nullable = true)
 |-- STOP: timestamp (nullable = true)
 |-- REASONCODE: string (nullable = true)
 |-- REASONDESCRIPTION: string (nullable = true)



encounter_id,patient_id,START,STOP,REASONCODE,REASONDESCRIPTION
44618b87-7d0a-5857-8ee8-24c0c8a5fa73,44618b87-7d0a-5857-1ced-875f976ab04c,2023-12-22T19:51:54.000Z,2023-12-23T19:51:54.000Z,183996000,Sterilization requested (situation)
7698543c-2438-424e-2881-aa30c990f7a9,7698543c-2438-424e-1697-e2a6e274e19a,2007-04-22T23:57:50.000Z,2007-04-30T12:47:50.000Z,67811000119102,Primary small cell malignant neoplasm of lung TNM stage 1 (disorder)
f39e7b28-8371-f78e-1fef-522cdced9ed6,f39e7b28-8371-f78e-f7e5-d6162c27a523,2021-06-02T22:31:32.000Z,2021-06-07T07:52:32.000Z,424132000,Non-small cell carcinoma of lung TNM stage 1 (disorder)
95c7d111-0d51-28dc-ceda-57954bc04796,95c7d111-0d51-28dc-664a-f70bd96e37df,2020-01-10T05:29:34.000Z,2020-01-14T13:35:34.000Z,424132000,Non-small cell carcinoma of lung TNM stage 1 (disorder)
27425c17-122d-99f6-94eb-c04cf1923bef,27425c17-122d-99f6-1459-ce4abe77c482,1989-02-08T22:30:31.000Z,1989-02-15T13:03:31.000Z,424132000,Non-small cell carcinoma of lung TNM stage 1 (disorder)
7b70301e-ebf0-7922-5794-f73e3b9d6769,7b70301e-ebf0-7922-db25-2471d16c1715,2014-09-18T16:11:31.000Z,2014-09-24T01:36:31.000Z,424132000,Non-small cell carcinoma of lung TNM stage 1 (disorder)
47c5e69e-e060-3c1a-9f4c-9525624aa646,47c5e69e-e060-3c1a-f260-b1365976c661,2003-03-30T07:11:16.000Z,2003-03-31T07:11:16.000Z,399261000,History of coronary artery bypass grafting (situation)
c8d0caae-d6b7-c986-3bca-c68240d73fbc,c8d0caae-d6b7-c986-7a4d-602f82f81dad,2017-02-26T02:17:53.000Z,2017-02-27T02:17:53.000Z,399261000,History of coronary artery bypass grafting (situation)
a3d00fca-1949-2558-f55c-7ffe8ad4b0d4,a3d00fca-1949-2558-cc4e-350ebe0dbad0,2008-02-06T09:12:23.000Z,2008-02-08T11:37:36.000Z,399261000,History of coronary artery bypass grafting (situation)
50532d01-0756-f5b4-5d9f-b4a6bad364ac,50532d01-0756-f5b4-42b6-de0c01ec3618,2019-10-02T14:01:17.000Z,2019-10-13T14:01:17.000Z,6525002,Dependent drug abuse (disorder)


**Verificacao da qualidade dos dados da tabela encouters_silver**

As verificações incluíram: duplicação de registros, presença de datas nulas, detecção de intervalos inválidos entre START e STOP, identificação de datas no futuro, análise do tempo de permanência (LOS), integridade referencial com patients_silver, e inspeção do campo de diagnóstico principal (REASONDESCRIPTION).

In [0]:
print("====================================================")
print("      QUALITY CHECK — encounters_silver")
print("====================================================")

# -----------------------------------------------------
# 1) SCHEMA
# -----------------------------------------------------
print("\n>> 1. Schema do encounters_silver:")
df_enc_silver.printSchema()


# -----------------------------------------------------
# 2) DUPLICATAS POR encounter_id
# -----------------------------------------------------
print("\n>> 2. Checando duplicatas por encounter_id:")

total = df_enc_silver.count()
unique = df_enc_silver.dropDuplicates(["encounter_id"]).count()

print(f"Total de registros:      {total}")
print(f"Registros únicos:        {unique}")
print(f"Duplicatas encontradas:  {total - unique}")


# -----------------------------------------------------
# 3) START E STOP NULOS
# -----------------------------------------------------
print("\n>> 3. Registros com START ou STOP nulos:")

null_times = df_enc_silver.filter(
    col("START").isNull() | col("STOP").isNull()
).count()

print(f"Registros com START/STOP nulos: {null_times}")


# -----------------------------------------------------
# 4) START/STOP NO FUTURO
# -----------------------------------------------------
print("\n>> 4. Registros com datas no futuro:")

start_future = df_enc_silver.filter(col("START") > current_timestamp()).count()
stop_future = df_enc_silver.filter(col("STOP") > current_timestamp()).count()

print(f"START no futuro: {start_future}")
print(f"STOP no futuro:  {stop_future}")


# -----------------------------------------------------
# 5) INTERVALOS INVÁLIDOS: STOP < START
# -----------------------------------------------------
print("\n>> 5. Internações com STOP < START (inválidas):")

invalid_intervals = df_enc_silver.filter(col("STOP") < col("START")).count()
print(f"Intervalos inválidos: {invalid_intervals}")


# -----------------------------------------------------
# 6) LOS ABSURDO: NEGATIVO OU LONGO DEMAIS
# -----------------------------------------------------
print("\n>> 6. LOS (Length of Stay) inválido ou exagerado:")

df_los = df_enc_silver.withColumn("los_days", datediff(col("STOP"), col("START")))

los_negative = df_los.filter(col("los_days") < 0).count()
los_over_365 = df_los.filter(col("los_days") > 365).count()  # mais de 1 ano

print(f"LOS negativo:          {los_negative}")
print(f"LOS > 365 dias:        {los_over_365}")


# -----------------------------------------------------
# 7) INTERVALO DE DATAS (min/max)
# -----------------------------------------------------
print("\n>> 7. Intervalo das datas (START e STOP):")

df_enc_silver.agg(
    spark_min("START").alias("min_start"),
    spark_max("START").alias("max_start"),
    spark_min("STOP").alias("min_stop"),
    spark_max("STOP").alias("max_stop")
).show()


# -----------------------------------------------------
# 8) INTEGRIDADE COM patients_silver
# -----------------------------------------------------
print("\n>> 8. Integridade referencial com patients_silver:")

missing_patients = (
    df_enc_silver.alias("e")
    .join(df_patients_silver.alias("p"),
          col("e.patient_id") == col("p.patient_id"),
          "left")
    .filter(col("p.patient_id").isNull())
    .count()
)

print(f"Internações sem paciente correspondente: {missing_patients}")


# -----------------------------------------------------
# 9) QUALIDADE DO DIAGNÓSTICO (REASONDESCRIPTION)
# -----------------------------------------------------
print("\n>> 9. Diagnóstico (REASONDESCRIPTION) — nulos e top 20:")

reason_nulls = df_enc_silver.filter(col("REASONDESCRIPTION").isNull()).count()
print(f"REASONDESCRIPTION nulos: {reason_nulls}")

print("\nTop 20 motivos de internação:")
display(
    df_enc_silver
        .groupBy("REASONDESCRIPTION")
        .count()
        .orderBy(col("count").desc())
        .limit(20)
)

print("\n### QUALITY CHECK — encounters_silver COMPLETO ###")


      QUALITY CHECK — encounters_silver

>> 1. Schema do encounters_silver:
root
 |-- encounter_id: string (nullable = true)
 |-- patient_id: string (nullable = true)
 |-- START: timestamp (nullable = true)
 |-- STOP: timestamp (nullable = true)
 |-- REASONCODE: string (nullable = true)
 |-- REASONDESCRIPTION: string (nullable = true)


>> 2. Checando duplicatas por encounter_id:
Total de registros:      8354
Registros únicos:        8354
Duplicatas encontradas:  0

>> 3. Registros com START ou STOP nulos:
Registros com START/STOP nulos: 0

>> 4. Registros com datas no futuro:
START no futuro: 0
STOP no futuro:  0

>> 5. Internações com STOP < START (inválidas):
Intervalos inválidos: 0

>> 6. LOS (Length of Stay) inválido ou exagerado:
LOS negativo:          0
LOS > 365 dias:        0

>> 7. Intervalo das datas (START e STOP):
+-------------------+-------------------+-------------------+-------------------+
|          min_start|          max_start|           min_stop|           max_sto

REASONDESCRIPTION,count
Non-small cell carcinoma of lung TNM stage 1 (disorder),2455
Dependent drug abuse (disorder),1117
Sterilization requested (situation),942
History of coronary artery bypass grafting (situation),645
Sleep disorder (disorder),389
Appendicitis (disorder),373
Chronic congestive heart failure (disorder),355
Primary small cell malignant neoplasm of lung TNM stage 1 (disorder),339
Awaiting transplantation of kidney (situation),253
Abnormal findings diagnostic imaging heart+coronary circulat (finding),224



### QUALITY CHECK — encounters_silver COMPLETO ###


**Observação sobre tratamento de inconsistências em encounters_silver**

As inconsistências identificadas foram tratadas conforme o papel da camada Silver no pipeline. Registros que violam regras básicas de integridade do modelo de dados (como internações sem paciente correspondente) foram removidos, assegurando consistência referencial. Campos categóricos ausentes, como o motivo da internação (`REASONDESCRIPTION`), foram padronizados com o valor `"Unknown"` exclusivamente para fins técnicos, sem imputação de significado clínico.

Registros com valores temporalmente plausíveis, porém semanticamente improváveis (ex.: datas de admissão muito antigas), foram mantidos nesta camada por rastreabilidade. A aplicação de filtros temporais e demais critérios analíticos será realizada explicitamente na camada Gold, de acordo com o escopo das análises do MVP.


In [0]:


print("====================================================")
print("   TRATAMENTO DE INCONSISTÊNCIAS — encounters_silver")
print("====================================================")

# Quantidade original de registros
original_count = df_enc_silver.count()
print(f"\nRegistros originais em df_enc_silver: {original_count}")

# -----------------------------------------------------
# 1) REMOVER INTERNAÇÕES SEM PACIENTE CORRESPONDENTE
# -----------------------------------------------------
print("\n>> 1. Removendo internações sem paciente correspondente:")

# pega apenas os patient_id válidos da patients_silver
valid_patients = df_patients_silver.select("patient_id").distinct()

# faz um INNER JOIN para manter apenas encounters com paciente existente
df_enc_silver_valid = (
    df_enc_silver
    .join(valid_patients, on="patient_id", how="inner")
)

valid_count = df_enc_silver_valid.count()
invalid_encounters = original_count - valid_count

print(f"Internações inválidas (sem paciente): {invalid_encounters}")
print(f"Registros após remoção:               {valid_count}")


# -----------------------------------------------------
# 2) TRATAR MOTIVO DE INTERNAÇÃO NULO
# -----------------------------------------------------
print("\n>> 2. Preenchendo REASONDESCRIPTION nulos com 'Unknown':")

missing_reason_before = df_enc_silver_valid.filter(col("REASONDESCRIPTION").isNull()).count()
print(f"Antes do tratamento: {missing_reason_before} registros com motivo nulo")

df_enc_silver_clean = df_enc_silver_valid.fillna({"REASONDESCRIPTION": "Unknown"})

missing_reason_after = df_enc_silver_clean.filter(col("REASONDESCRIPTION") == "Unknown").count()
print(f"Depois do tratamento: {missing_reason_after} registros marcados como 'Unknown'")


# -----------------------------------------------------
# Resultado final
# -----------------------------------------------------
final_count = df_enc_silver_clean.count()
print("\n### TRATAMENTO DE INCONSISTÊNCIAS — COMPLETO ###")
print(f"Total final de registros limpos: {final_count}")


   TRATAMENTO DE INCONSISTÊNCIAS — encounters_silver

Registros originais em df_enc_silver: 8354

>> 1. Removendo internações sem paciente correspondente:
Internações inválidas (sem paciente): 2
Registros após remoção:               8352

>> 2. Preenchendo REASONDESCRIPTION nulos com 'Unknown':
Antes do tratamento: 1 registros com motivo nulo
Depois do tratamento: 1 registros marcados como 'Unknown'

### TRATAMENTO DE INCONSISTÊNCIAS — COMPLETO ###
Total final de registros limpos: 8352


## Persistência dos dados na camada Silver

Os DataFrames resultantes foram persistidos na camada **Silver** como **Delta Tables** (`patients_silver`, `encounters_silver` e `conditions_silver`). Essas tabelas representam a versão padronizada e tecnicamente consistente dos dados provenientes da camada Bronze, incorporando checagens básicas de qualidade e integridade referencial, e servem como base direta para a modelagem dimensional e para as transformações analíticas da camada Gold.


In [0]:
%sql
USE CATALOG mvp_engenharia_de_dados;
USE SCHEMA silver;


In [0]:
df_patients_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("patients_silver")


In [0]:
df_enc_silver_clean.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("encounters_silver")


## Metadados (catálogo): comentários por coluna

Nesta seção, são registrados comentários de coluna nas tabelas Silver usando `COMMENT ON COLUMN`.
Isso facilita exploração, documentação e governança mínima do catálogo no contexto do MVP.


In [0]:
columns_comments_patients = {
    "patient_id": "Identificador único do paciente",
    "BIRTHDATE": "Data de nascimento do paciente",
    "DEATHDATE": "Data de falecimento do paciente",
    "GENDER": "Gênero",
    "RACE": "Raça",
    "ETHNICITY": "Etnia"
}

for col, comment in columns_comments_patients.items():
    spark.sql(f"COMMENT ON COLUMN patients_silver.{col} IS '{comment}'")

In [0]:
columns_comments_encounters = {
    "encounter_id": "Identificador único do encontro",
    "patient_id": "Identificador do paciente",
    "START": "Data/hora de início do encontro",
    "STOP": "Data/hora de término do encontro",
    "REASONCODE": "Código do motivo do encontro",
    "REASONDESCRIPTION": "Descrição do motivo do encontro"
}

for col, comment in columns_comments_encounters.items():
    spark.sql(f"COMMENT ON COLUMN encounters_silver.{col} IS '{comment}'")

In [0]:
%sql
-- Metadados da tabela encounters_silver
DESCRIBE TABLE EXTENDED silver.encounters_silver;

col_name,data_type,comment
patient_id,string,Identificador do paciente
encounter_id,string,Identificador único do encontro
START,timestamp,Data/hora de início do encontro
STOP,timestamp,Data/hora de término do encontro
REASONCODE,string,Código do motivo do encontro
REASONDESCRIPTION,string,Descrição do motivo do encontro
,,
# Delta Statistics Columns,,
Column Names,"encounter_id, STOP, START, REASONCODE, patient_id, REASONDESCRIPTION",
Column Selection Method,first-32,


In [0]:
%sql
-- Metadados da tabela patients_silver
DESCRIBE TABLE EXTENDED silver.patients_silver;

col_name,data_type,comment
patient_id,string,Identificador único do paciente
BIRTHDATE,date,Data de nascimento do paciente
DEATHDATE,date,Data de falecimento do paciente
GENDER,string,Gênero
RACE,string,Raça
ETHNICITY,string,Etnia
,,
# Delta Statistics Columns,,
Column Names,"DEATHDATE, RACE, GENDER, BIRTHDATE, ETHNICITY, patient_id",
Column Selection Method,first-32,


## Conclusão da camada Silver

A camada Silver entrega dados:
- Padronizados
- Rastreáveis
- Com integridade mínima garantida

Essas tabelas servem como base direta para a modelagem dimensional e o cálculo dos indicadores hospitalares na camada Gold.
